# Converting JSON data into a tabular, ball-by-ball format

In [21]:
import pandas as pd
import json
import os
from datetime import datetime
import copy

In [22]:
def load_data_from_json(filepath):
    dict_format = json.loads(open(filepath, "r").read())
    return dict_format

In [23]:
class MatchSituation:
    def __init__(self, match, delivery=None, init=False, previous_delivery = None, format=50):
        self.previous_delivery = previous_delivery
        self.next_delivery = None
        self.parent = match

        if init:
            self.bowlers = {
                x: {"balls_remaining": 6*format/5, "finished_spell": False, "at_crease": False, "wickets": 0, "runs": 0} for x in match.team1_players
            }
            self.batters = {
                x: {"at_crease": False, "on_strike": False, "dismissed": False, "runs": 0, "balls": 0} for x in match.team2_players
            }
            self.runs_remaining = match.match['target']['runs']
            self.wickets_remaining = 10
            self.balls_remaining = 6*format
            self.current_runs = 0
            self.current_wickets = 0
            self.current_balls = 0
            self.extras_count = 0
            self.batter_on_strike = None
            self.bowler_on_strike = None
            self.runs = 0
        else:
            previous_delivery.next_delivery = self
            self.bowlers = copy.deepcopy(previous_delivery.bowlers)
            self.batters = copy.deepcopy(previous_delivery.batters)
            self.runs_remaining = copy.deepcopy(previous_delivery.runs_remaining)
            self.wickets_remaining = copy.deepcopy(previous_delivery.wickets_remaining)
            self.balls_remaining = copy.deepcopy(previous_delivery.balls_remaining)
            self.current_runs = copy.deepcopy(previous_delivery.current_runs)
            self.current_wickets = copy.deepcopy(previous_delivery.current_wickets)
            self.current_balls = copy.deepcopy(previous_delivery.current_balls)
            self.extras_count = copy.deepcopy(previous_delivery.extras_count)
            self.batter_on_strike = copy.deepcopy(previous_delivery.batter_on_strike)
            self.bowler_on_strike = copy.deepcopy(previous_delivery.bowler_on_strike)
            self.update(delivery)
        
    def update(self, delivery):
        # print(delivery)
        
        self.batters[delivery['batter']]['at_crease'] = True
        self.batters[delivery['batter']]['on_strike'] = True
        self.batter_on_strike = delivery['batter']
        self.batters[delivery['non_striker']]['at_crease'] = True
        self.batters[delivery['non_striker']]['on_strike'] = False
        
        self.batters[delivery['batter']]['runs'] += delivery['runs']['batter']
        self.current_runs += delivery['runs']['total']
        self.runs_remaining -= delivery['runs']['total']

        self.runs = delivery['runs']['total']

        self.extras_count += delivery['runs']['extras']

        if ('extras' not in delivery):
            self.current_balls += 1

        if (('extras' in delivery) and ('wides' in delivery['extras'] or 'noballs' in delivery['extras'])) or ('extras' not in delivery):
            self.bowlers[delivery['bowler']]['runs'] += delivery['runs']['total']

        if (('extras' in delivery) and ('wides' in delivery['extras'])) == False:

            self.batters[delivery['batter']]['balls'] += 1

            if (('extras' in delivery) and ('noballs' in delivery['extras'])) == False:

                self.bowlers[delivery['bowler']]['balls_remaining'] -= 1
                self.bowlers[delivery['bowler']]['at_crease'] = True
                self.bowler_on_strike = delivery['bowler']
                self.balls_remaining -= 1
                self.current_balls += 1

            if self.bowlers[delivery['bowler']]['balls_remaining'] % 6 == 0:
                self.bowlers[delivery['bowler']]['at_crease'] = False

            if self.bowlers[delivery['bowler']]['balls_remaining'] == 0:
                self.bowlers[delivery['bowler']]['finished_spell'] = True
        
        if 'wickets' in delivery:
            self.wickets_remaining -= 1
            self.current_wickets += 1

            self.batters[delivery['wickets'][0]['player_out']]['dismissed'] = True
            self.batters[delivery['wickets'][0]['player_out']]['at_crease'] = False
            if delivery['wickets'][0]['kind'] in ['caught', "bowled", "lbw", "stumped"]:
                self.bowlers[delivery['bowler']]['wickets'] += 1

        

class OneDayMatch:
    def __init__(self, match_dict, matchno=None):
        self.match_date = datetime.strptime(match_dict['info']['dates'][0], "%Y-%m-%d")
        self.team1 = match_dict['innings'][0]['team']
        self.team2 = match_dict['innings'][1]['team']
        self.toss_winner = match_dict['info']['toss']['winner']
        self.ground = match_dict['info']['venue']
        self.team1_players = match_dict['info']['players'][self.team1]
        self.team2_players = match_dict['info']['players'][self.team2]
        self.match = match_dict['innings'][1]
        self.winner = match_dict['info']['outcome']['winner']
        self.matchno = matchno
        self.ball_by_ball = [MatchSituation(self, format=50, init=True)]
        self.process_match()
    
    def process_match(self):
        for over in self.match['overs']:
            for delivery in over['deliveries']:
                self.ball_by_ball.append(MatchSituation(
                    self, delivery=delivery, previous_delivery=self.ball_by_ball[-1]
                ))


In [24]:
def dump(obj):
  for attr in dir(obj):
    print("obj.%s = %r" % (attr, getattr(obj, attr)))

domestic_loc = '../../data/step_02/domestic/'
domestic_matches = []

for filename in [x for x in os.listdir(domestic_loc) if x.endswith('.json')]:
  try:
    match_dict = load_data_from_json(domestic_loc + filename)
    match = OneDayMatch(match_dict, matchno=filename)
  except KeyError as e:
    continue
  domestic_matches.append(match)

international_loc = '../../data/step_02/international/'
international_matches = []

for filename in [x for x in os.listdir(international_loc) if x.endswith('.json')]:
  try:
    match_dict = load_data_from_json(international_loc + filename)
    match = OneDayMatch(match_dict, matchno=filename)
  except KeyError as e:
    continue
  international_matches.append(match)

In [ ]:
import os

big_df = pd.DataFrame(columns=['runs_remaining', 'wickets_remaining', 'balls_remaining', 'current_runs', 'batter_on_strike', 'bowler_on_strike', 'outcome'])

for match in domestic_matches + international_matches:
    for situation in match.ball_by_ball[1:]:
        situation_dict = {
            "runs_remaining": situation.runs_remaining,
            "wickets_remaining": situation.wickets_remaining,
            "balls_remaining": situation.balls_remaining,
            "current_runs": situation.current_runs,
            "batter_on_strike": situation.batter_on_strike,
            "bowler_on_strike": situation.bowler_on_strike,
            "outcome": situation.runs
        }
        big_df.loc[len(big_df)] = situation_dict
    os.makedirs("../../data/step_03", exist_ok=True)
    big_df.to_csv("../../data/step_03/one_day_matches.csv", index=False)